# Introduction of new Python Package/Tool
---
Installation notes and any related comments...

In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas-profiling

     ------------------------------------- 324.4/324.4 kB 10.1 MB/s eta 0:00:00
     ------------------------------------- 357.9/357.9 kB 21.7 MB/s eta 0:00:00
     ------------------------------------- 666.2/666.2 kB 20.5 MB/s eta 0:00:00
     ------------------------------------- 300.0/300.0 kB 18.1 MB/s eta 0:00:00
     ---------------------------------------- 296.5/296.5 kB ? eta 0:00:00
     ---------------------------------------- 102.7/102.7 kB ? eta 0:00:00
     ------------------------------------- 395.2/395.2 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 4.7/4.7 MB 23.3 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 19.9 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27092 sha256=0cd41ff9ec0f132ddc6cdf3fc4dfe4f7d78a905f5122b83fa6b3c9ac2b17d97c
  Stored in directory: c:\users\angus

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.6.3 which is incompatible.


In [7]:
# All import statements 
# Packages Explored in Module
import pandas as pd 
import numpy
import matplotlib

# New Tool for This Project
import pandas_profiling as pp

# Individually written tools from module files

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.6/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.6/u/import-error

# The Evolution of Musical Attributes and Their Impact on the Popularity of Songs in the Digital Era

### Problem Statement:
In the age of digital music, the success of a song is often gauged by its popularity on streaming platforms, which can be influenced by various musical attributes. Understanding how different attributes correlate with popularity can provide insights into listener preferences and inform decisions in the music production and marketing processes.

### Objective:

To analyze the relationship between various song attributes such as genre, tempo, and explicit content and their popularity scores.
To identify trends in the evolution of musical attributes over time and how they relate to shifts in listener engagement and popularity.
To determine if there are specific patterns or characteristics that are common among highly popular songs.

### Hypotheses:

Songs with certain attributes, like a higher tempo or inclusion of explicit content, are more likely to achieve high popularity scores.
Changes in listener preferences over time will reflect in the prominence of different musical attributes within the most popular songs.
There may be a pattern of convergence or divergence in song attributes within top-charting hits compared to songs with lower popularity scores.

### Research Questions:

What song attributes most strongly correlate with high popularity scores within the dataset?
How have the common attributes of popular songs changed over the past decade?
Can we predict the potential popularity of a song based on its attributes without considering its audio features?

### Significance:
The project's findings can offer valuable insights for artists, producers, and record labels into the types of song attributes that resonate with listeners in the current music landscape. Additionally, these insights could be used to develop more targeted music recommendation systems that align with evolving listener preferences.

# 2. Collect Data
---

dataset is MusicOSet

In [2]:
# Code to read in data

# 3. Exploratory Data Analysis
---
Comments...

In [3]:
# Exploration code

In [4]:
# Figures

Descriptions and discoveries...

In [5]:
# Exploration code

In [6]:
# Figures

Descriptions and discoveries...

In [7]:
# Exploration code (potential integration of module code)

In [8]:
# Figures (potential integration of module code)

Descriptions and discoveries...

# 4. Data Pre-Processing
---
Explanations...

In [9]:
# Data manipulation 1 (potential integration of module code)

In [10]:
# Data manipulation 2 (potential integration of module code)

Related discussion...

# 5. In-Depth Analysis
---
Explanations...

In [11]:
# Implementations (potential integration of module code)

Descriptions and discoveries...

In [12]:
# Visualisations (potential integration of module code)

Descriptions and discoveries...

In [13]:
# Evaluations (potential integration of module code)

Descriptions and discoveries...

In [14]:
# Visualisations (potential integration of module code)

Descriptions and discoveries...

# 6. Communicate Results
---
Overall conclusions...

# References
---
...

# Group Reflection

---

Comments...

---
*Optional Unequal Contribution Table*
Member | Contribution [% effort]
-|-
Jamie Soan | 100%
Angus Day | 100%
William Heap | 100%
kj | 100%

# Jamie Reflection
---
Comments...

# Angus Reflection
---
Comments...

# William Reflection
---
Comments...

# KJ Reflection
---
Comments...

# JKL Reflection
---
Comments...